In [1]:
#coding:utf-8
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 常用模型预测
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error,make_scorer

from math import isnan
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from scipy import stats

In [2]:
# 载入数据
train = pd.read_csv("train.csv")
test = pd.read_csv("test_noLabel.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ID        196539 non-null  int64  
 1   位置        196508 non-null  float64
 2   出租方式      24230 non-null   float64
 3   区         196508 non-null  float64
 4   卧室数量      196539 non-null  int64  
 5   卫的数量      196539 non-null  int64  
 6   厅的数量      196539 non-null  int64  
 7   地铁站点      91778 non-null   float64
 8   地铁线路      91778 non-null   float64
 9   小区名       196539 non-null  int64  
 10  小区房屋出租数量  195538 non-null  float64
 11  居住状态      20138 non-null   float64
 12  总楼层       196539 non-null  float64
 13  房屋朝向      196539 non-null  object 
 14  房屋面积      196539 non-null  float64
 15  时间        196539 non-null  int64  
 16  楼层        196539 non-null  int64  
 17  装修情况      18492 non-null   float64
 18  距离        91778 non-null   float64
 19  Label     196539 non-null  float64
dtypes: f

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56279 entries, 0 to 56278
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        56279 non-null  int64  
 1   位置        56269 non-null  float64
 2   出租方式      4971 non-null   float64
 3   区         56269 non-null  float64
 4   卧室数量      56279 non-null  int64  
 5   卫的数量      56279 non-null  int64  
 6   厅的数量      56279 non-null  int64  
 7   地铁站点      26494 non-null  float64
 8   地铁线路      26494 non-null  float64
 9   小区名       56279 non-null  int64  
 10  小区房屋出租数量  56257 non-null  float64
 11  居住状态      4483 non-null   float64
 12  总楼层       56279 non-null  float64
 13  房屋朝向      56279 non-null  object 
 14  房屋面积      56279 non-null  float64
 15  时间        56279 non-null  int64  
 16  楼层        56279 non-null  int64  
 17  装修情况      4207 non-null   float64
 18  距离        26494 non-null  float64
dtypes: float64(11), int64(7), object(1)
memory usage: 8.2+ MB


In [5]:
# 在之前的缺失值分析中，地铁线路、地铁站点和距离缺失比例相同，进一步观察可知三者缺失位置也完全一致，可以猜测缺失原因为该房源周围没有地铁线路
# 将缺失的地铁线路填充为0
train['地铁线路'] = train['地铁线路'].fillna(0)
test['地铁线路'] = test['地铁线路'].fillna(0)
train['地铁站点'] = train['地铁站点'].fillna(0)
test['地铁站点'] = test['地铁站点'].fillna(0)

In [6]:
orientation_headers = ['东', '南', '西', '北',
                       '东南', '西南', '西北', '东北']
def fill_orientation(item, orientation):
    x = item.split(' ')
    return 1 if orientation in x else 0
 
for i in orientation_headers:
    train[i] = train['房屋朝向'].apply(lambda x: fill_orientation(x, i))

for i in orientation_headers:
    test[i] = test['房屋朝向'].apply(lambda x: fill_orientation(x, i))
    
train.drop('房屋朝向', axis=1, inplace=True)
test.drop('房屋朝向', axis=1, inplace=True)

In [7]:
# 进行排序后，使用前一条数据对小区房屋出租数量进行填充
train = train.sort_values(by=['小区名', '楼层', '时间'], ascending=(True, True, True))
test = test.sort_values(by=['小区名', '楼层', '时间'], ascending=(True, True, True))

train['小区房屋出租数量'] = train['小区房屋出租数量'].fillna(method='pad')
test['小区房屋出租数量'] = test['小区房屋出租数量'].fillna(method='pad')

In [8]:
# 发现数据中没有5，预测缺失值为该小区
train['区'] = train['区'].fillna(5)
test['区'] = test['区'].fillna(5)
# 位置中缺少76
train['位置'] = train['位置'].fillna(76)
test['位置'] = test['位置'].fillna(76)

In [9]:
data = pd.concat([train, test], axis=0, ignore_index=True)

In [10]:
xiaoqu_dis = data.groupby('小区名')['距离'].mean()
dict_xiaoqu_dis = {'小区名':xiaoqu_dis.index,'平均距离':xiaoqu_dis.values}
df_xiaoqu_dis = pd.DataFrame(dict_xiaoqu_dis)

data = data.merge(df_xiaoqu_dis, on='小区名',how='left')
data['距离'] = data['距离'].fillna(data['平均距离'])

In [11]:
xiaqu_sub_line = data.groupby('小区名')['地铁线路'].max()
dict_xiaqu_sub_line = {'小区名':xiaqu_sub_line.index,'小区地铁线路':xiaqu_sub_line.values}
df_xiaqu_sub_line = pd.DataFrame(dict_xiaqu_sub_line)

data = data.merge(df_xiaqu_sub_line, on='小区名',how='left')
data['地铁线路'] = data['小区地铁线路']

In [12]:
xiaqu_sub = data.groupby('小区名')['地铁站点'].max()
dict_xiaqu_sub = {'小区名':xiaqu_sub.index,'小区地铁站点':xiaqu_sub.values}
df_xiaqu_sub = pd.DataFrame(dict_xiaqu_sub)
data = data.merge(df_xiaqu_sub, on='小区名',how='left')
data['地铁站点'] = data['小区地铁站点']

In [13]:
data.drop(['平均距离','小区地铁线路','小区地铁站点'],axis=1,inplace=True)

In [14]:
data['距离'] = data['距离'].fillna(0)
data['居住状态'] = data['居住状态'].fillna(0)
data['装修情况'] = data['装修情况'].fillna(0)
data['出租方式'] = data['出租方式'].fillna(2)

In [15]:
# 构造新的特征
data['房间总数'] = data['卫的数量'] + data['卧室数量'] + data['厅的数量']
data['卧和卫'] = data['卫的数量'] + data['卧室数量']
data['卧和厅'] = data['卧室数量'] + data['厅的数量']
data['卧室均面积'] = data['房屋面积'] * 0.3 / (data['卧室数量'] + 1)
data['卫均面积'] = data['房屋面积'] * 0.06 / (data['卫的数量'] + 1)
data['厅均面积'] = data['房屋面积'] * 0.25 / (data['厅的数量'] + 1)
data['卧室总面积'] = data['卧室均面积'] * data['卧室数量']
data['除卧室外面积'] = data['房屋面积'] - data['卧室总面积']
data['厅总面积'] = data['厅均面积'] * data['厅的数量']
data['楼层比'] = (data['楼层'] + 1) / data['总楼层']

# 每个楼层的卧室面积
temp = data.groupby('楼层')['卧室总面积'].sum().reset_index()
temp.columns = ['楼层','楼层卧室总面积']
data = data.merge(temp, how = 'left',on = '楼层')

# 每个楼层的房屋面积
temp = data.groupby('楼层')['房屋面积'].sum().reset_index()
temp.columns = ['楼层','楼层房屋面积']
data = data.merge(temp, how = 'left',on = '楼层')

# 每个小区附近的地铁站点数
temp = data.groupby('小区名')['地铁站点'].count().reset_index()
temp.columns = ['小区名','地铁站点数量']
data = data.merge(temp, how = 'left',on = '小区名')

# 每个位置附近的地铁站点数
temp = data.groupby('位置')['地铁站点'].count().reset_index()
temp.columns = ['位置','商圈地铁站点数量']
data = data.merge(temp, how = 'left',on = '位置')

# 每个小区出租房源平均房屋面积
area_mean = data.groupby('小区名')['房屋面积'].mean().reset_index()
area_mean.columns = ['小区名','小区房屋平均面积']
data = data.merge(area_mean, how = 'left',on = '小区名')

# 每个位置附近的小区数
temp = data.groupby('位置')['小区名'].count().reset_index()
temp.columns = ['位置','商圈小区数量']
data = data.merge(temp, how = 'left',on = '位置')

# 每个小区平均房间数
#temp = data.groupby('小区名')['房间总数'].count().reset_index()
#temp.columns = ['小区名','region_room_total']
#data = data.merge(temp, how = 'left',on = '小区名')

In [16]:
data.head()

,ID,位置,出租方式,区,卧室数量,卫的数量,厅的数量,地铁站点,地铁线路,小区名,...,卧室总面积,除卧室外面积,厅总面积,楼层比,楼层卧室总面积,楼层房屋面积,地铁站点数量,商圈地铁站点数量,小区房屋平均面积,商圈小区数量
0,107337,59.0,2.0,6.0,4,1,2,5.7,3.0,0,...,47.269116,149.685535,32.825775,0.366667,1.957894e+06,9.327799e+06,1,1315,196.954651,1315
1,3620,59.0,2.0,6.0,2,1,2,5.7,3.0,1,...,27.143330,108.573320,22.619442,0.091667,2.593948e+06,1.255755e+07,6,1315,179.852146,1315
2,129171,59.0,2.0,6.0,2,1,2,5.7,3.0,1,...,27.143330,108.573320,22.619442,0.091667,2.593948e+06,1.255755e+07,6,1315,179.852146,1315
3,4688,59.0,2.0,6.0,4,2,1,5.7,3.0,1,...,48.460775,153.459119,25.239987,0.275000,2.382085e+06,1.139076e+07,6,1315,179.852146,1315
4,86482,59.0,2.0,6.0,7,2,2,5.7,3.0,1,...,53.003972,148.915922,33.653316,0.275000,2.382085e+06,1.139076e+07,6,1315,179.852146,1315


In [17]:
qu_rent = data.groupby('区')['Label'].mean()
dict_qu_rent = {'区':qu_rent.index,'qu_rent':qu_rent.values}
df_qu_rent = pd.DataFrame(dict_qu_rent)
df_qu_rent['qu_rent'] = df_qu_rent['qu_rent'].rank()
data = data.merge(df_qu_rent, on='区',how='left')

In [18]:
data.head()

,ID,位置,出租方式,区,卧室数量,卫的数量,厅的数量,地铁站点,地铁线路,小区名,...,除卧室外面积,厅总面积,楼层比,楼层卧室总面积,楼层房屋面积,地铁站点数量,商圈地铁站点数量,小区房屋平均面积,商圈小区数量,qu_rent
0,107337,59.0,2.0,6.0,4,1,2,5.7,3.0,0,...,149.685535,32.825775,0.366667,1.957894e+06,9.327799e+06,1,1315,196.954651,1315,14.0
1,3620,59.0,2.0,6.0,2,1,2,5.7,3.0,1,...,108.573320,22.619442,0.091667,2.593948e+06,1.255755e+07,6,1315,179.852146,1315,14.0
2,129171,59.0,2.0,6.0,2,1,2,5.7,3.0,1,...,108.573320,22.619442,0.091667,2.593948e+06,1.255755e+07,6,1315,179.852146,1315,14.0
3,4688,59.0,2.0,6.0,4,2,1,5.7,3.0,1,...,153.459119,25.239987,0.275000,2.382085e+06,1.139076e+07,6,1315,179.852146,1315,14.0
4,86482,59.0,2.0,6.0,7,2,2,5.7,3.0,1,...,148.915922,33.653316,0.275000,2.382085e+06,1.139076e+07,6,1315,179.852146,1315,14.0


In [19]:
df_train = data[data.Label.notna()].copy()
df_test = data[data.Label.isna()].copy()

df_train.shape, df_test.shape

((196539, 44), (56279, 44))

In [20]:
df_train_asc = pd.DataFrame()
df_train_asc[['ID','Label','east','west','south','north','east_north','east-south','west_north','west_south','loc','rent_way',
             'region','bedroom_num','toilet_num','drawing_num','sub_station','sub_line','region_name','region_house_num',
             'living_condition','total_floor','house_area','time','floor','deco','distance','room_num','bed_toi','bed_dra',
             'bed_mean','toi_mean','dra_mean','bed_total','exc_bed','dra_total','floor_ratio','floor_bed','floor_house',
             'reg_sub_station_num','loc_sub_station_num','reg_house_mean','loc_reg_num','qu_rent'
             ]] = df_train[['ID','Label','东', '西', '南', '北', '东北', '东南', '西北', '西南', '位置',
                                                     '出租方式', '区',  '卧室数量', '卫的数量', '厅的数量', '地铁站点', 
                                                     '地铁线路', '小区名', '小区房屋出租数量', '居住状态', '总楼层',
                                                     '房屋面积', '时间', '楼层', '装修情况', '距离', '房间总数', '卧和卫', 
                                                     '卧和厅','卧室均面积', '卫均面积', '厅均面积', '卧室总面积', '除卧室外面积',
                                                     '厅总面积', '楼层比', '楼层卧室总面积','楼层房屋面积', '地铁站点数量', 
                                                     '商圈地铁站点数量', '小区房屋平均面积', '商圈小区数量','qu_rent']]

In [21]:
df_test_asc = pd.DataFrame()
df_test_asc[['ID','Label','east','west','south','north','east_north','east-south','west_north','west_south','loc','rent_way',
             'region','bedroom_num','toilet_num','drawing_num','sub_station','sub_line','region_name','region_house_num',
             'living_condition','total_floor','house_area','time','floor','deco','distance','room_num','bed_toi','bed_dra',
             'bed_mean','toi_mean','dra_mean','bed_total','exc_bed','dra_total','floor_ratio','floor_bed','floor_house',
             'reg_sub_station_num','loc_sub_station_num','reg_house_mean','loc_reg_num','qu_rent'
             ]] = df_test[['ID','Label','东', '西', '南', '北', '东北', '东南', '西北', '西南', '位置',
                                                     '出租方式', '区',  '卧室数量', '卫的数量', '厅的数量', '地铁站点', 
                                                     '地铁线路', '小区名', '小区房屋出租数量', '居住状态', '总楼层',
                                                     '房屋面积', '时间', '楼层', '装修情况', '距离', '房间总数', '卧和卫', 
                                                     '卧和厅','卧室均面积', '卫均面积', '厅均面积', '卧室总面积', '除卧室外面积',
                                                     '厅总面积', '楼层比', '楼层卧室总面积','楼层房屋面积', '地铁站点数量', 
                                                     '商圈地铁站点数量', '小区房屋平均面积', '商圈小区数量','qu_rent']]

In [22]:
model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves=900,
                              learning_rate=0.02, n_estimators=5100, bagging_fraction=0.7,
                              feature_fraction=0.6, reg_alpha=0.5, reg_lambda=5,
                              min_data_in_leaf=18, min_sum_hessian_in_leaf=0.001,
                             )

In [23]:
#df_train_asc.drop('time',axis=1, inplace=True)
#df_test_asc.drop('time',axis=1, inplace=True)

In [24]:
df_train_asc.columns

Index(['ID', 'Label', 'east', 'west', 'south', 'north', 'east_north',
       'east-south', 'west_north', 'west_south', 'loc', 'rent_way', 'region',
       'bedroom_num', 'toilet_num', 'drawing_num', 'sub_station', 'sub_line',
       'region_name', 'region_house_num', 'living_condition', 'total_floor',
       'house_area', 'time', 'floor', 'deco', 'distance', 'room_num',
       'bed_toi', 'bed_dra', 'bed_mean', 'toi_mean', 'dra_mean', 'bed_total',
       'exc_bed', 'dra_total', 'floor_ratio', 'floor_bed', 'floor_house',
       'reg_sub_station_num', 'loc_sub_station_num', 'reg_house_mean',
       'loc_reg_num', 'qu_rent'],
      dtype='object')

In [25]:
feature_cols = ['east', 'west', 'south', 'north', 'east_north',
       'east-south', 'west_north', 'west_south', 'loc', 'rent_way', 'region',
       'bedroom_num', 'toilet_num', 'drawing_num', 'sub_station', 'sub_line',
       'region_name', 'region_house_num', 'living_condition', 'total_floor',
       'house_area', 'time', 'floor', 'deco', 'distance', 'room_num',
       'bed_toi', 'bed_dra', 'bed_mean', 'toi_mean', 'dra_mean', 'bed_total',
       'exc_bed', 'dra_total', 'floor_ratio', 'floor_bed', 'floor_house',
       'reg_sub_station_num', 'loc_sub_station_num', 'reg_house_mean',
       'loc_reg_num', 'qu_rent']

In [26]:
# 划分数据集
X_data = df_train_asc[feature_cols]
Y_data = df_train_asc['Label']

x_train,x_val,y_train,y_val = train_test_split(X_data,Y_data,test_size=0.3)

In [27]:
X_test = df_test_asc[feature_cols]

In [28]:
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data)) 

In [29]:
model_lgb.fit(x_train, y_train)
val_lgb = model_lgb.predict(x_val)
MSE_lgb = mean_squared_error(y_val,val_lgb)
print('MSE of val with lgb:',MSE_lgb)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
MSE of val with lgb: 1.6796353104588895


In [30]:
print('Predict lgb...')
model_lgb_pre = model_lgb.fit(X_data,Y_data)
subA_lgb = model_lgb_pre.predict(X_test)
print('Sta of Predict lgb:')
Sta_inf(subA_lgb)

Predict lgb...
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
Sta of Predict lgb:
_min 0.07649931620168984
_max: 93.43551425856224
_mean 8.050800349376424
_ptp 93.35901494236055
_std 6.256130385463831
_var 39.13916739992383


In [31]:
statistic_pred = pd.read_csv('statistic_pred.csv')

In [32]:
sub_lgb = pd.DataFrame()
sub_lgb['ID'] = test.ID
sub_lgb['Label'] = subA_lgb

In [33]:
sub_lgb.head(10)

,ID,Label
12575,209114,3.419172
641,197180,2.170379
9959,206498,4.031785
6047,202586,5.476869
8613,205152,5.518007
26862,223401,5.460312
10258,206797,5.282046
46569,243108,5.902109
2219,198758,14.603984
3239,199778,10.984722


In [34]:
sub_lgb = sub_lgb.sort_values(by=['ID'])

In [35]:
statistic_pred['Label_x'] = sub_lgb['Label']

In [36]:
statistic_pred.head()

,ID,Label,Label_x
0,196539,4.244482,4.282213
1,196540,NaN,6.119181
2,196541,NaN,13.429661
3,196542,NaN,5.971799
4,196543,4.923599,4.958421


In [37]:
statistic_pred['Label_y'] = statistic_pred['Label'].fillna(statistic_pred['Label_x'])

In [38]:
sub_lgb_sta = statistic_pred[['ID','Label_y']]

In [39]:
sub_lgb_sta['Label'] = sub_lgb_sta['Label_y']
sub_lgb_sta.drop(['Label_y'],axis=1,inplace=True)

In [40]:
sub_lgb_sta

,ID,Label
0,196539,4.244482
1,196540,6.119181
2,196541,13.429661
3,196542,5.971799
4,196543,4.923599
...,...,...
56274,252813,4.923599
56275,252814,13.000663
56276,252815,7.457767
56277,252816,13.073005


In [42]:
sub_lgb_sta.to_csv("sub_lgb_sta.csv",index=False)